In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, Embedding, Layer, MultiHeadAttention, Add, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
# Configure how graphs will show up in this notebook
%matplotlib inline
seaborn.set_context('notebook', rc={'figure.figsize': (10, 6)}, font_scale=1.5)

In [ ]:
def load_embeddings(filename):
    """
    Load a DataFrame from the generalized text format used by word2vec, GloVe,
    fastText, and ConceptNet Numberbatch. The main point where they differ is
    whether there is an initial line with the dimensions of the matrix.
    """
    labels = []
    rows = []
    with open(filename, encoding='utf-8') as infile:
        for i, line in enumerate(infile):
            items = line.rstrip().split(' ')
            if len(items) == 2:
                # This is a header row giving the shape of the matrix
                continue
            labels.append(items[0])
            values = np.array([float(x) for x in items[1:]], 'f')
            rows.append(values)
    
    arr = np.vstack(rows)
    return pd.DataFrame(arr, index=labels, dtype='f')

embeddings = load_embeddings('')
EMBED_SIZE = embeddings.shape[1]

print(f'Loaded {len(embeddings)} embeddings of size {EMBED_SIZE}')

In [6]:
reviews = pd.read_csv('IMDB_Dataset.csv')

# print number of reviews
print('Number of reviews: {}'.format(len(reviews)))

# print average review length
print('Average review length: {}'.format(np.mean([len(x) for x in reviews['review'].str.split()])))

# print 95th percentile review length
print('90th percentile review length: {}'.format(np.percentile([len(x) for x in reviews['review'].str.split()], 90)))

Number of reviews: 50000
Average review length: 231.15694
90th percentile review length: 451.0


In [ ]:
X = reviews['review']
y = reviews['sentiment']

In [ ]:
NUM_TOP_WORDS = None # use entire vocabulary!
MAX_LEN = 512 # maximum and minimum number of words
NUM_CLASSES = 2

tokenizer = Tokenizer(
                    num_words=NUM_TOP_WORDS,
                    filters = '—!"“”#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t…\'‘’'
                     )

tokenizer.fit_on_texts(X)

sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index
NUM_TOP_WORDS = len(word_index) if NUM_TOP_WORDS==None else NUM_TOP_WORDS
top_words = min((len(word_index),NUM_TOP_WORDS))

X = pad_sequences(sequences, maxlen=MAX_LEN)

y = keras.utils.to_categorical(y, num_classes=NUM_CLASSES)

print(f"Found {len(word_index):,} unique tokens. Distilled to {top_words:,} top words.")



# now fill in the matrix, using the ordering from the
# keras word tokenizer from before
found_words = 0
embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))

for word, i in word_index.items():
    
    try:
        embedding_vector = embeddings.loc[word]
        # words not found in embedding index will be ALL-ZEROS
        embedding_matrix[i] = embedding_vector
        found_words = found_words+1
    except:
        #print(word)
        pass


print(f"Embedding Shape: {embedding_matrix.shape}")
print(f"Total words found: {found_words:,}")
print(f"Percentage: {round(100 * found_words / embedding_matrix.shape[0], 2)}")

In [ ]:
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBED_SIZE,
                            weights=[embedding_matrix],# here is the embedding getting saved
                            input_length=MAX_LEN,
                            trainable=False)

class PositionalEncoding(Layer):
    def __init__(self, max_len, embedding_dim, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.max_len = max_len
        self.embedding_dim = embedding_dim

    def call(self, inputs):
        positions = tf.range(start=0, limit=self.max_len, delta=1)
        positions = tf.cast(positions, tf.float32)
        positions = tf.expand_dims(positions, axis=-1)
        pos_encoding = inputs + (positions / 10000 ** (2 * (positions // 2) / self.embedding_dim))
        return pos_encoding

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
num_heads = 8  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer
num_trans = 3

inputs = Input(shape=(MAX_LEN,), dtype=tf.int32)
embedding = embedding_layer(inputs)
pos_encoding = PositionalEncoding(MAX_LEN, EMBED_SIZE)
x = pos_encoding(embedding)

transformer_block = TransformerBlock(EMBED_SIZE, num_heads, ff_dim)

for i in range(num_trans): 
    x = transformer_block(x) 
    
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.1)(x)

outputs = Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=10)